In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pickle
import os

In [2]:
orders = pd.read_csv('instacart_2017_05_01/order_products__prior.csv')

In [4]:
def construct_dic_matrix(df):
    order_groups = orders.groupby('order_id')
    matrix = {}
    for order_id in order_groups.groups.keys():
        cart = order_groups.get_group(order_id)['product_id'].sort_values().to_list()
        for i, el in enumerate(cart):
            if el not in matrix:
                matrix[el] = {}
            for j in cart[(i+1):]:
                matrix[el][j] = matrix[el].get(j, 0) + 1

    delkeys = []
    for i in matrix.keys():
        for k,v in matrix[i].items():
            if v == 1:
                delkeys.append((i,k))

    for i,j in delkeys:
        del matrix[i][j]

    return matrix

In [5]:
def to_sparseMatrix(matrix):
    mat = sp.dok_matrix((49689 , 49689), dtype=np.int16)

    for ix1, ixx2 in matrix.items():
        for ix2 in ixx2:
            mat[ix1, ix2] = ixx2[ix2]

    mat = mat.transpose().tocsr()
    return mat

In [6]:
%%time
MATRIX = construct_dic_matrix(orders)

CPU times: user 52min 24s, sys: 17.8 s, total: 52min 42s
Wall time: 52min 31s


In [0]:
with open('output/MATRIX_DICT.pkl', 'wb') as f:
    pickle.dump(MATRIX, f)

In [6]:
with open('output/MATRIX_DICT.pkl', 'rb') as f:
    MATRIX = pickle.load(f)

In [5]:
len(MATRIX)

49677

In [9]:
sparse_matrix = to_sparseMatrix(MATRIX)

In [13]:
with open('SPARSE_MATRIX.pkl', 'wb') as f:
    pickle.dump(sparse_matrix, f)

NameError: name 'sparse_matrix' is not defined

<49689x49689 sparse matrix of type '<class 'numpy.int16'>'
	with 18804842 stored elements in Compressed Sparse Row format>

In [5]:
sparse_matrix

NameError: name 'sparse_matrix' is not defined